8 puzzle problem

In [ ]:
import heapq

# A class to represent the puzzle state
class PuzzleState:
    def _init_(self, state, parent=None, move=None, g=0, h=0):
        self.state = state
        self.parent = parent
        self.move = move
        self.g = g  # cost to reach this state
        self.h = h  # heuristic value
        self.f = g + h  # total cost (f = g + h)
        
    # Compare states for priority in the open list (min-heap)
    def _lt_(self, other):
        return self.f < other.f

# Goal state for the puzzle
goal_state = [
    [2, 8, 1],
    [0, 4, 3],
    [7, 6, 5]
]

# Directions for moving the blank space: Up, Down, Left, Right
DIRECTIONS = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # (row, col)

# Function to find the position of the blank space (represented by 0)
def find_blank(state):
    for i in range(3):
        for j in range(3):
            if state[i][j] == 0:
                return i, j
    return -1, -1

# Function to compute the Manhattan distance heuristic
def manhattan_distance(state):
    distance = 0
    for i in range(3):
        for j in range(3):
            value = state[i][j]
            if value != 0:
                goal_x, goal_y = (value - 1) // 3, (value - 1) % 3
                distance += abs(i - goal_x) + abs(j - goal_y)
    return distance

# Function to check if a state is the goal state
def is_goal_state(state):
    return state == goal_state

# Function to generate valid neighbors by moving the blank space
def generate_neighbors(state):
    blank_x, blank_y = find_blank(state)
    neighbors = []
    
    for dx, dy in DIRECTIONS:
        new_x, new_y = blank_x + dx, blank_y + dy
        if 0 <= new_x < 3 and 0 <= new_y < 3:
            # Create a new state by swapping the blank space with the adjacent value
            new_state = [row[:] for row in state]
            new_state[blank_x][blank_y], new_state[new_x][new_y] = new_state[new_x][new_y], new_state[blank_x][blank_y]
            neighbors.append(new_state)
    return neighbors

# A* algorithm to solve the 8-puzzle problem
def a_star(initial_state):
    open_list = []
    closed_list = set()
    start_state = PuzzleState(initial_state, g=0, h=manhattan_distance(initial_state))
    heapq.heappush(open_list, start_state)
    
    while open_list:
        current_state = heapq.heappop(open_list)
        
        # If we reach the goal state, return the solution path
        if is_goal_state(current_state.state):
            path = []
            while current_state.parent:
                path.append(current_state.move)
                current_state = current_state.parent
            path.reverse()
            return path
        
        closed_list.add(tuple(tuple(row) for row in current_state.state))
        
        for neighbor in generate_neighbors(current_state.state):
            if tuple(tuple(row) for row in neighbor) not in closed_list:
                g = current_state.g + 1
                h = manhattan_distance(neighbor)
                neighbor_state = PuzzleState(neighbor, parent=current_state, move=neighbor, g=g, h=h)
                heapq.heappush(open_list, neighbor_state)
    
    return None  # No solution found

# Initial state
initial_state = [
    [1, 2, 3],
    [8, 0, 4],
    [7, 6, 5]
]

# Solve the puzzle
solution = a_star(initial_state)
if solution:
    print("Solution path:")
    for move in solution:
        for row in move:
            print(row)
        print()
else:
    print("No solution found")